# ++ Carregar dados históricos da Bovespa em Dataframe para análise ++

In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Tue Aug 30 12:57:08 2022'

In [2]:
import sys
from pandas import Series
import pandas as pd
from datetime import date
pd.__version__

'1.3.4'

In [3]:
import matplotlib as mat
mat.__version__

'3.4.3'

In [4]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
from IPython.display import FileLink, FileLinks
FileLink('Manipulação CSV BB.ipynb')

In [5]:
# Setando opção do Pandas para mostrar todas as linhas
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
# vr_investimento_padrao = 20000

In [8]:
def carrega_dados():
    layout = [2, 8, 2, 12, 3, 12, 10, 3, 4, 13, 13, 13, 13, 13, 13, 13, 5, 18, 18, 13, 1, 8, 7, 13, 12, 3]
#     df = pd.read_fwf("arquivos/COTAHIST_A2021.ZIP", widths=layout, compression='zip',engine='c',
#                     names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

    df = pd.read_fwf("arquivos/COTAHIST_A2022.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

################## temporário #####################
    df_112021 = pd.read_fwf("arquivos/COTAHIST_M112021.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

    df_122021 = pd.read_fwf("arquivos/COTAHIST_M122021.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

    df = pd.concat([df_112021, df_122021, df])

################## temporário #####################

    df = df.loc[df["CODBDI"] == 2]
    
    df02 = pd.DataFrame({
         "cdAcao":df["CODNEG"]
        ,"dtPregao":pd.to_datetime(df["DTEXCH"], format="%Y%m%d", errors="ignore")
        ,"vrFech":df["PREULT"] / 100
        ,"vrVolume":df["VOLTOT"] / 100
        ,"vrMaxDia":df["PREMAX"] / 100
        ,"vrMinDia":df["PREMIN"] / 100
        ,"vrAbert":df["PREABE"] / 100
    })
    df02 = df02.sort_values(["cdAcao", "dtPregao"], ascending=True)
    
    df02["pcVar"], df02["pcMaxDia"], df02["pcMinDia"], df02["pcAbert"] = [
         ((df02.vrFech   / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMaxDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMinDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrAbert  / df02.vrFech.shift(1)) -1) *100
        ]
    
    df02["ic05"], df02["ic10"], df02["ic15"], df02["ic20"], df02["ic25"], df02["ic30"] = [
         df02.apply(condicao05, axis=1)
        ,df02.apply(condicao10, axis=1)
        ,df02.apply(condicao15, axis=1)
        ,df02.apply(condicao20, axis=1)
        ,df02.apply(condicao25, axis=1)
        ,df02.apply(condicao30, axis=1)
        ]

    # Filtrando Ação
    # df02.loc[df02["cdAcao"]=="AMBP3"].sort_values(["cdAcao", "dtPregao"], ascending=False)
    
    return df02

In [9]:
def condicao05(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 0.5) else 0

In [10]:
def condicao10(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1) else 0

In [11]:
def condicao15(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1.5) else 0

In [12]:
def condicao20(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2) else 0

In [13]:
def condicao25(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2.5) else 0

In [14]:
def condicao30(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 3) else 0

In [15]:
# Exclui header e trailer
# df = df.drop(df.index[[0, -1]])

In [16]:
df = carrega_dados()

In [17]:
# Seleciona últimos 25 dias de cada ação
df25d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[24])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df20d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[19])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df15d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[14])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df10d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[9])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df5d  = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[4])].sort_values(["cdAcao", "dtPregao"], ascending=False)

In [18]:
df05 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol = df25d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech = df25d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")
dfPcAbert = df25d.groupby("cdAcao")["pcAbert"].agg("mean").reset_index(name="pcAbert")

In [19]:
df05_5d = df5d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_5d = df5d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_5d = df5d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_5d = df5d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_5d = df5d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_5d = df5d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_5d = df5d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_5d = df5d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")
dfPcAbert_5d = df5d.groupby("cdAcao")["pcAbert"].agg("mean").reset_index(name="pcAbert")

In [20]:
df05_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_10d = df10d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_10d = df10d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")
dfPcAbert_10d = df10d.groupby("cdAcao")["pcAbert"].agg("mean").reset_index(name="pcAbert")

In [21]:
df05_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_15d = df15d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_15d = df15d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")
dfPcAbert_15d = df15d.groupby("cdAcao")["pcAbert"].agg("mean").reset_index(name="pcAbert")

In [22]:
df05_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_20d = df20d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_20d = df20d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")
dfPcAbert_20d = df20d.groupby("cdAcao")["pcAbert"].agg("mean").reset_index(name="pcAbert")

In [23]:
df05["1.0%"], df05["1.5%"], df05["2.0%"], df05["2.5%"], df05["3.0%"] , df05["AvgVol"], df05["AvgVrFech"], df05["AvgPcAbert"] = [
    df10["resultado"],df15["resultado"],df20["resultado"],df25["resultado"],df30["resultado"],dfVol["vol"],dfVrFech["vrFech"],dfPcAbert["pcAbert"]]
df05_5d["1.0%"], df05_5d["1.5%"], df05_5d["2.0%"], df05_5d["2.5%"], df05_5d["3.0%"] , df05_5d["AvgVol"], df05_5d["AvgVrFech"], df05_5d["AvgPcAbert"] = [
    df10_5d["resultado"],df15_5d["resultado"],df20_5d["resultado"],df25_5d["resultado"],df30_5d["resultado"],dfVol_5d["vol"],dfVrFech_5d["vrFech"],dfPcAbert_5d["pcAbert"]]
df05_10d["1.0%"], df05_10d["1.5%"], df05_10d["2.0%"], df05_10d["2.5%"], df05_10d["3.0%"] , df05_10d["AvgVol"], df05_10d["AvgVrFech"], df05_10d["AvgPcAbert"] = [
    df10_10d["resultado"],df15_10d["resultado"],df20_10d["resultado"],df25_10d["resultado"],df30_10d["resultado"],dfVol_10d["vol"],dfVrFech_10d["vrFech"],dfPcAbert_10d["pcAbert"]]
df05_15d["1.0%"], df05_15d["1.5%"], df05_15d["2.0%"], df05_15d["2.5%"], df05_15d["3.0%"] , df05_15d["AvgVol"], df05_15d["AvgVrFech"], df05_15d["AvgPcAbert"] = [
    df10_15d["resultado"],df15_15d["resultado"],df20_15d["resultado"],df25_15d["resultado"],df30_15d["resultado"],dfVol_15d["vol"],dfVrFech_15d["vrFech"],dfPcAbert_15d["pcAbert"]]
df05_20d["1.0%"], df05_20d["1.5%"], df05_20d["2.0%"], df05_20d["2.5%"], df05_20d["3.0%"] , df05_20d["AvgVol"], df05_20d["AvgVrFech"], df05_20d["AvgPcAbert"] = [
    df10_20d["resultado"],df15_20d["resultado"],df20_20d["resultado"],df25_20d["resultado"],df30_20d["resultado"],dfVol_20d["vol"],dfVrFech_20d["vrFech"],dfPcAbert_20d["pcAbert"]]

In [24]:
df05     =     df05.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_5d  =  df05_5d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_10d = df05_10d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_15d = df05_15d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_20d = df05_20d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)

In [25]:
pd.set_option('display.max_rows', None)
# df06 = df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 20) & (df05["AvgVrFech"] > 5)]

In [26]:
df25d['vrVolume'] = df25d['vrVolume'].astype(int)
df25d['vrVolume'] = df25d['vrVolume'].map('{:,}'.format)

### 25 dias, order by 1%

In [27]:
df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 22) & (df05["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
962,PGMN3,24,24,20,18,16,11,"8,445,852.88",5.01,0.36
974,POSI3,23,23,21,20,18,15,"26,870,743.88",9.05,0.13
831,MILS3,24,22,21,19,15,14,"11,193,420.96",8.45,0.17
1092,SEQL3,22,22,20,20,19,18,"20,981,330.00",6.87,0.28
68,AGRO3,22,22,20,16,11,7,"14,691,965.32",25.56,0.16
773,LJQQ3,22,22,18,17,14,13,"20,054,343.32",7.12,0.06
578,FESA4,23,22,18,14,12,10,"9,488,879.64",49.03,0.35
615,GGPS3,22,22,17,13,11,7,"27,503,168.04",13.39,0.16


### 20 dias, order by 1%

In [28]:
df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["1.0%"] >= 18) & (df05_20d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
933,PGMN3,19,19,15,14,13,9,"9,686,178.90",5.17,0.29
989,RECV3,18,18,17,17,16,11,"58,623,274.30",25.05,0.11
67,AGRO3,18,18,17,14,10,6,"15,392,179.15",26.03,0.18
83,AMER3,19,18,17,12,10,10,"538,929,795.25",15.25,1.08
1055,SEQL3,18,18,16,16,15,14,"21,945,837.25",7.22,0.31
944,POSI3,18,18,16,16,14,13,"31,143,220.10",9.64,0.11
89,ARML3,18,18,16,14,13,12,"6,326,708.45",15.07,0.19
750,LJQQ3,18,18,15,14,11,10,"21,550,135.00",7.43,0.05
565,FESA4,19,18,15,11,9,8,"9,584,301.20",49.88,0.35
601,GGPS3,18,18,14,10,8,4,"31,377,990.50",13.64,0.19


### 15 dias, order by 1%

In [29]:
df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["1.0%"] >= 14) & (df05_15d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
80,AMER3,14,14,14,9,7,7,"601,219,535.07",15.55,1.35
912,PTBL3,14,14,11,10,8,8,"6,685,106.87",9.68,0.09
896,PGMN3,14,14,10,9,8,4,"11,006,965.60",5.22,0.16


### 10 dias, order by 1%

In [30]:
df05_10d.loc[(df05_10d["AvgVol"] > 6000000) & (df05_10d["1.0%"] >= 9) & (df05_10d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
74,AMER3,9,9,9,7,5,5,"669,082,235.20",16.34,1.48
895,RECV3,9,9,8,8,7,5,"88,271,579.40",25.34,-0.01
839,PETR4,9,9,7,7,5,2,"2,662,439,747.10",33.08,0.05
844,PGMN3,9,9,7,6,6,3,"8,356,318.70",5.22,0.18
73,AMBP3,9,9,7,6,4,3,"26,232,791.00",27.37,-0.09
860,PTBL3,9,9,6,5,4,4,"6,486,684.50",9.69,-0.21


### 5 dias, order by 1%

In [31]:
df05_5d.loc[(df05_5d["AvgVol"] > 6000000) & (df05_5d["2.0%"] >= 5) & (df05_5d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
366,CVCB3,5,5,5,5,5,4,"128,524,034.40",7.92,0.16
73,ANIM3,5,5,5,5,4,4,"16,944,857.20",5.00,-0.01
625,LJQQ3,5,5,5,5,4,4,"13,877,950.60",7.07,0.23
826,RECV3,5,5,5,5,4,4,"137,401,441.00",25.93,0.04
879,SBFG3,5,5,5,5,4,4,"45,825,383.40",21.43,0.03
677,MILS3,5,5,5,5,4,3,"14,507,241.60",9.35,0.14
55,AGRO3,5,5,5,5,4,2,"20,309,816.20",28.35,0.12
662,MATD3,5,5,5,5,4,2,"6,460,998.20",9.41,-0.16
965,TTEN3,5,5,5,5,4,2,"23,063,665.20",10.73,0.21
1042,YDUQ3,5,5,5,5,4,2,"53,489,607.80",12.68,0.40


### Consulta ação específica

In [46]:
df25d.loc[(df25d["cdAcao"] == "CVCB3")].replace(0, "")

,cdAcao,dtPregao,vrFech,vrVolume,vrMaxDia,vrMinDia,vrAbert,pcVar,pcMaxDia,pcMinDia,pcAbert,ic05,ic10,ic15,ic20,ic25,ic30
1093623,CVCB3,2022-08-29,8.22,"87,394,219",8.37,7.80,7.86,2.62,4.49,-2.62,-1.87,1,1,1,1,1,1
1091931,CVCB3,2022-08-26,8.01,"103,714,159",8.43,7.77,8.29,-2.55,2.55,-5.47,0.85,1,1,1,1,1,
1090247,CVCB3,2022-08-25,8.22,"155,462,851",8.48,8.05,8.10,2.88,6.13,0.75,1.38,1,1,1,1,1,1
1088557,CVCB3,2022-08-24,7.99,"198,351,293",8.08,7.18,7.18,11.28,12.53,,,1,1,1,1,1,1
1086867,CVCB3,2022-08-23,7.18,"97,697,650",7.20,6.90,6.99,3.16,3.45,-0.86,0.43,1,1,1,1,1,1
1085202,CVCB3,2022-08-22,6.96,"66,107,937",7.03,6.79,7.03,-1.83,-0.85,-4.23,-0.85,,,,,,
1083507,CVCB3,2022-08-19,7.09,"91,409,840",7.46,7.01,7.36,-5.59,-0.67,-6.66,-2.00,,,,,,
1081824,CVCB3,2022-08-18,7.51,"99,997,913",7.91,7.34,7.89,-3.84,1.28,-6.02,1.02,1,1,,,,
1080148,CVCB3,2022-08-17,7.81,"100,394,418",8.04,7.55,7.63,0.13,3.08,-3.21,-2.18,1,1,1,1,1,1
1078475,CVCB3,2022-08-16,7.80,"82,495,345",8.02,7.58,7.86,-0.38,2.43,-3.19,0.38,1,1,1,1,,


In [35]:
### 25 dias, Order by 0.5% # df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5% # df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5% # df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

## Definindo valores para compra

In [36]:
# Valor disponível para cmp / 1000
# Valor da ação no leilão
vr_corretagem = 9.8
emolumentos = 0.000325
acao = 'LIGT3'
vr_disp = 110
vr_acao_leilao = 1148 / 100
vr_qtd = round((vr_disp * 1000) / (vr_acao_leilao * 100)) * 100

In [37]:
vr_cmp = round(vr_acao_leilao * 1.014, 2)

vr_tot_cmp = vr_acao_leilao * vr_qtd
corret_cmp = vr_corretagem + (vr_tot_cmp * emolumentos)
vr_tot_liq_cmp = vr_tot_cmp + corret_cmp

vr_acao_vnd_10 = float('{0:.2f}'.format(vr_acao_leilao * 1.0108 + 0.01))
vr_tot_vnd_10 = vr_acao_vnd_10 * vr_qtd
corret_vnd_10 = vr_corretagem + (vr_tot_vnd_10 * emolumentos)
vr_tot_liq_vnd_10 = vr_tot_vnd_10 - corret_vnd_10

vr_acao_vnd_15 = float('{0:.2f}'.format(vr_acao_leilao * 1.0158 + 0.01))
vr_tot_vnd_15 = vr_acao_vnd_15 * vr_qtd
corret_vnd_15 = vr_corretagem + (vr_tot_vnd_15 * emolumentos)
vr_tot_liq_vnd_15 = vr_tot_vnd_15 - corret_vnd_15

In [38]:
d = {'Compra':     ['Quantidade', 'Valor Leilão', 'Total', 'Total líquido', 'Corretagem Compra'],
     ' ':          [vr_qtd, vr_acao_leilao, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
     'Venda 1,0%': ['Valor compra', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,0%':       [vr_cmp, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
     'Venda 1,5%': [' ', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,5%':       [' ', vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
    }
df_calc = pd.DataFrame(data=d)
df_calc

,Compra,,"Venda 1,0%","1,0%","Venda 1,5%","1,5%"
0,Quantidade,"9,600.00",Valor compra,11.64,,
1,Valor Leilão,11.48,Valor,11.61,Valor,11.67
2,Total,"110,208.00",Total,"111,456.00",Total,"112,032.00"
3,Total líquido,"110,253.62",Total líquido,"111,409.98",Total líquido,"111,985.79"
4,Corretagem Compra,45.62,Corretagem Venda,46.02,Corretagem Venda,46.21


In [39]:
d2 = {'Index':      ['Quantidade', 'Valor', 'Total', 'Total líquido', 'Corretagem'],
      'Compra':     [vr_qtd, vr_cmp, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
      'Venda 1.0%': [vr_qtd, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
      'Venda 1.5%': [vr_qtd, vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
     }
df_calc2 = pd.DataFrame(data=d2)
df_calc2.set_index('Index')
df_calc2

,Index,Compra,Venda 1.0%,Venda 1.5%
0,Quantidade,"9,600.00","9,600.00","9,600.00"
1,Valor,11.64,11.61,11.67
2,Total,"110,208.00","111,456.00","112,032.00"
3,Total líquido,"110,253.62","111,409.98","111,985.79"
4,Corretagem,45.62,46.02,46.21


In [40]:
time.asctime( time.localtime(time.time()) )

'Tue Aug 30 12:58:32 2022'